# 🍽️ Zomato CSAO — LLM-Powered Dataset Generator

**Pipeline:**
1. Load Qwen2.5-14B-Instruct on the A100
2. Generate a unique, realistic menu for each of 200 restaurants via LLM
3. LLM also produces restaurant-specific combo rules (what goes with what)
4. Code-based simulation generates 600K+ cart session rows using those menus
5. Export 5 CSVs ready for LightGBM ranker training

**Runtime estimate on Colab A100:** ~25-35 mins (menu gen) + ~2 mins (simulation)

## ⚙️ Step 0 — Install & imports

In [ ]:
!pip install -q transformers accelerate bitsandbytes sentencepiece pyarrow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 26.2 MB/s eta 0:00:00


In [2]:
import torch
import json
import re
import os
import time
import random
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.notebook import tqdm

np.random.seed(42)
random.seed(42)

print(f'GPU: {torch.cuda.get_device_name(0)}')
print(f'VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

GPU: NVIDIA A100-SXM4-80GB
VRAM: 85.1 GB


## 🤖 Step 1 — Load Qwen2.5-14B-Instruct

In [3]:
MODEL_ID = 'Qwen/Qwen2.5-14B-Instruct'

print(f'Loading {MODEL_ID}...')
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,   # BF16 — fits in 40GB VRAM
    device_map='auto',
    trust_remote_code=True,
)
model.eval()
print('Model loaded.')
print(f'VRAM used: {torch.cuda.memory_allocated()/1e9:.1f} GB')

Loading Qwen/Qwen2.5-14B-Instruct...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/579 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Model loaded.
VRAM used: 29.5 GB


## 🏗️ Step 2 — Define restaurant profiles (no LLM needed here)

In [14]:
CITIES = ['Mumbai', 'Delhi', 'Bengaluru', 'Hyderabad', 'Pune', 'Chennai', 'Kolkata']
N_RESTAURANTS = 100

# Each cuisine type has realistic restaurant name pools and a personality descriptor
# The personality string goes into the LLM prompt to give each restaurant a unique flavour
CUISINE_PROFILES = {
    'biryani': {
        'names': ['Behrouz Biryani', 'Paradise', 'Bawarchi', 'Hyderabadi Zaika', 'Dum Pukht',
                  'Biryani Blues', 'Royal Biryani', 'Nawab Biryani', 'Lucknowi Dawat', 'Biryani by Kilo'],
        'personalities': [
            'traditional Hyderabadi dum biryani specialist with Mughal heritage',
            'modern biryani chain with fusion variants and bowl formats',
            'Lucknowi-style Awadhi biryani known for delicate spicing',
            'coastal biryani specialist with prawn and fish variants',
            'budget-friendly neighbourhood biryani with generous portions',
        ],
        'price_range': (180, 420),
    },
    'north_indian': {
        'names': ['Punjab Grill', 'Moti Mahal', 'Dhaba Express', 'Haveli', 'Tandoor House',
                  'Pind Balluchi', 'Barbeque Nation', 'Gulati', 'Sagar Ratna', 'Kake Da Hotel'],
        'personalities': [
            'upscale Punjabi fine dining with tandoor specialties',
            'rustic highway-style dhaba with homestyle curries',
            'family restaurant focused on dal and roti combos',
            'Rajasthani thali specialist with laal maas and bajra roti',
            'kebab-forward North Indian with Mughlai influence',
        ],
        'price_range': (150, 500),
    },
    'south_indian': {
        'names': ['Saravana Bhavan', 'Dosa Plaza', 'Murugan Idli', 'Annapoorna', 'Udupi Palace',
                  'MTR', 'Woodlands', 'Vaango', 'Sarvana Stores', 'A2B'],
        'personalities': [
            'traditional Tamil Brahmin restaurant with filter coffee focus',
            'dosa specialist with 30+ varieties including paper and set dosas',
            'Udupi-style vegetarian with coconut-forward Mangalorean dishes',
            'Chettinad cuisine specialist with bold spicing',
            'Kerala-focused with appam, stew and seafood specialties',
        ],
        'price_range': (80, 300),
    },
    'pizza': {
        'names': ['Pizza Hut', 'Dominos', 'La Pino Z', 'Oven Story', 'Slice of Italy',
                  'Pizza Express', 'The Pizza Company', 'Smoky Joe', 'Indie Pizza', 'Nazz Pizza'],
        'personalities': [
            'NY-style thin crust pizza with classic American toppings',
            'gourmet artisan pizza with wood-fired oven and Italian ingredients',
            'Indian-fusion pizza with paneer tikka and tandoori chicken toppings',
            'budget pizza chain focused on value combos and meal deals',
            'Chicago deep-dish specialist with thick crust and extra cheese',
        ],
        'price_range': (150, 600),
    },
    'burgers': {
        'names': ['Burger Singh', 'The Burger Lab', 'Smash Bros', 'Bun & Patty', 'Carl Jr',
                  'Shake Shack', 'Burgrill', 'Johnny Rockets', 'The Fatty Bao', 'Wat-a-Burger'],
        'personalities': [
            'Indian-spiced burger chain with desi masala patties',
            'smash burger specialist with crispy edges and special sauce',
            'gourmet burger bar with wagyu patties and craft beer pairings',
            'fast-casual burger with fresh ingredients and customizable builds',
            'nostalgic American diner-style burgers with thick shakes',
        ],
        'price_range': (120, 450),
    },
    'chinese': {
        'names': ['Mainland China', 'Wok Express', 'Dragon Palace', 'Orient Bites', 'Golden Dragon',
                  'Yo China', 'China Garden', 'The Great Wall', 'Panda Wok', 'Lao She'],
        'personalities': [
            'Indo-Chinese street food focused on hakka noodles and manchurian',
            'authentic Sichuan restaurant with mapo tofu and dan dan noodles',
            'Cantonese dim sum specialist with weekend brunch trolleys',
            'quick-service Chinese with customizable wok boxes',
            'pan-Asian restaurant blending Chinese Thai and Japanese flavours',
        ],
        'price_range': (120, 400),
    },
}

PRICE_TIERS = {'budget': 0.7, 'mid': 1.0, 'premium': 1.5}

# Generate restaurant metadata (no LLM needed — purely structural)
random.seed(42)
restaurants = []
cuisines = list(CUISINE_PROFILES.keys())
for i in range(N_RESTAURANTS):
    cuisine = cuisines[i % len(cuisines)]  # distribute evenly
    profile = CUISINE_PROFILES[cuisine]
    tier = random.choice(['budget', 'mid', 'mid', 'premium'])  # mid is most common
    city = random.choice(CITIES)
    personality = random.choice(profile['personalities'])
    lo, hi = profile['price_range']
    tmult = PRICE_TIERS[tier]
    restaurants.append({
        'restaurant_id': f'R{i:04d}',
        'name': f"{random.choice(profile['names'])} - {city}",
        'city': city,
        'cuisine': cuisine,
        'price_tier': tier,
        'personality': personality,
        'price_lo': int(lo * tmult),
        'price_hi': int(hi * tmult),
        'rating': round(random.uniform(3.2, 4.9), 1),
        'monthly_orders': random.randint(200, 5000),
    })

restaurants_df = pd.DataFrame(restaurants)
print(f'Created {len(restaurants_df)} restaurant profiles')
print(restaurants_df.groupby(['cuisine','price_tier']).size().unstack(fill_value=0))

Created 100 restaurant profiles
price_tier    budget  mid  premium
cuisine                           
biryani            7    6        4
burgers            4    6        6
chinese            2   11        3
north_indian       5    7        5
pizza              3    7        7
south_indian       6   10        1


## 🧠 Step 3 — LLM menu generation (one call per restaurant)

In [15]:
SYSTEM_PROMPT = """You are a food data expert for an Indian food delivery app.
You output ONLY raw valid JSON with no markdown, no code fences, no explanation.
Do not write anything before or after the JSON object."""

MENU_PROMPT_TEMPLATE = """
Generate a realistic menu for this Indian restaurant:

Restaurant: {name}
Cuisine type: {cuisine}
Personality: {personality}
Price tier: {tier} (price range Rs.{price_lo}-{price_hi} per item)
City: {city}

Return a JSON object with exactly two keys:

1. "items": a list of 45-55 menu items. Each item must have:
   - "name": string (unique dish name specific to this restaurant)
   - "category": one of ["main", "side", "drink", "dessert", "snack"]
   - "cuisine_tag": sub-cuisine string (e.g. "biryani", "north_indian", "beverages")
   - "price": integer in INR within the price range above
   - "is_veg": boolean
   - "calories": integer approximate
   - "popularity_score": float between 0.3 and 1.0

2. "combo_rules": a list of 4-8 objects each with keys:
   - "if_category": string
   - "if_cuisine_tag": string
   - "then_category": string
   - "then_cuisine_tag": string
   - "weight": float between 0.3 and 0.8

Mix: roughly 35% mains, 25% sides, 20% drinks, 15% desserts, 5% snacks.
Output the JSON object only. No other text.
"""

def extract_last_json(text: str):
    last_close = text.rfind('}')
    if last_close == -1:
        return None
    depth = 0
    for i in range(last_close, -1, -1):
        if text[i] == '}':   depth += 1
        elif text[i] == '{': depth -= 1
        if depth == 0:
            try:
                return json.loads(text[i:last_close + 1])
            except json.JSONDecodeError:
                return None
    return None


def generate_menu(restaurant: dict):
    prompt = MENU_PROMPT_TEMPLATE.format(
        name=restaurant['name'],
        cuisine=restaurant['cuisine'],
        personality=restaurant['personality'],
        tier=restaurant['price_tier'],
        price_lo=restaurant['price_lo'],
        price_hi=restaurant['price_hi'],
        city=restaurant['city'],
    )
    messages = [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'user',   'content': prompt},
    ]
    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = tokenizer(text, return_tensors='pt').to(model.device)
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=2500,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )
    new_tokens = output_ids[0][inputs['input_ids'].shape[1]:]
    raw = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    return extract_last_json(raw)


def validate_menu(data: dict) -> bool:
    """Basic sanity check on LLM output."""
    if not isinstance(data, dict): return False
    items = data.get('items', [])
    if len(items) < 5: return False
    required = {'name', 'category', 'cuisine_tag', 'price', 'is_veg', 'calories', 'popularity_score'}
    for item in items:
        if not required.issubset(item.keys()): return False
        if item['category'] not in ['main', 'side', 'drink', 'dessert', 'snack']:
            item['category'] = 'main'   # coerce rather than reject
    return True


print('Helper functions ready.')

Helper functions ready.


In [16]:
# Test on one restaurant
test_rest = restaurants_df.iloc[5].to_dict()
result = generate_menu(test_rest)

if result and validate_menu(result):
    print(f"✅ SUCCESS — {test_rest['name']} ({test_rest['cuisine']}, {test_rest['price_tier']})")
    print(f"   {len(result['items'])} items | {len(result.get('combo_rules', []))} combo rules")
    print("\nItems:")
    for it in result['items']:
        veg = '🟢' if it['is_veg'] else '🔴'
        print(f"  {veg} [{it['category']:7}] {it['name']:40} ₹{it['price']}")
    print("\nCombo rules:")
    for r in result.get('combo_rules', []):
        print(f"  {r['if_category']}({r['if_cuisine_tag']}) → {r['then_category']}({r['then_cuisine_tag']}) [w={r['weight']}]")
else:
    print("❌ FAILED")

✅ SUCCESS — China Garden - Mumbai (chinese, mid)
   42 items | 5 combo rules

Items:
  🟢 [main   ] Hakka Noodles Special                    ₹250
  🔴 [main   ] Chicken Manchurian                       ₹280
  🟢 [main   ] Veg Hakka Fried Rice                     ₹220
  🟢 [side   ] Spring Rolls                             ₹120
  🟢 [drink  ] Green Tea                                ₹80
  🟢 [snack  ] Fruit Chaat                              ₹100
  🟢 [drink  ] Mango Lassi                              ₹120
  🟢 [dessert] Gulab Jamun                              ₹150
  🟢 [main   ] Paneer Manchurian                        ₹260
  🔴 [main   ] Egg Fried Rice                           ₹200
  🟢 [side   ] Vegetable Spring Rolls                   ₹110
  🟢 [drink  ] Iced Coffee                              ₹100
  🟢 [dessert] Pineapple Popsicle                       ₹90
  🔴 [main   ] Chicken Chilli                           ₹270
  🟢 [main   ] Hakka Noodles                            ₹230
  🟢 [side   ] Tom

In [ ]:
# ─── MAIN GENERATION LOOP ───────────────────────────────────────────────────
# Saves progress to disk every 10 restaurants so you can resume if Colab crashes

SAVE_PATH = 'generate_menu_new.json'
MAX_RETRIES = 2

# Load existing progress if resuming
if os.path.exists(SAVE_PATH):
    with open(SAVE_PATH) as f:
        generated_menus = json.load(f)
    print(f'Resuming — already have {len(generated_menus)} menus')
else:
    generated_menus = {}

failed = []
start_time = time.time()

for _, rest in tqdm(restaurants_df.iterrows(), total=len(restaurants_df), desc='Generating menus'):
    rid = rest['restaurant_id']
    if rid in generated_menus:
        continue  # already done

    result = None
    for attempt in range(MAX_RETRIES):
        try:
            result = generate_menu(rest.to_dict())
            if result and validate_menu(result):
                break
            print(f'  [WARN] {rid} attempt {attempt+1} produced invalid output, retrying...')
        except Exception as e:
            print(f'  [ERROR] {rid}: {e}')

    if result and validate_menu(result):
        generated_menus[rid] = result
    else:
        failed.append(rid)
        print(f'  [FAIL] {rid} — will use fallback')

    # Save checkpoint every 10 restaurants
    if len(generated_menus) % 10 == 0:
        with open(SAVE_PATH, 'w') as f:
            json.dump(generated_menus, f)

# Final save
with open(SAVE_PATH, 'w') as f:
    json.dump(generated_menus, f)

elapsed = (time.time() - start_time) / 60
print(f'\nDone. {len(generated_menus)} menus generated, {len(failed)} failed.')
print(f'Elapsed: {elapsed:.1f} mins | Avg: {elapsed/max(len(generated_menus),1)*60:.1f}s per restaurant')

Generating menus:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
# ─── FALLBACK for failed restaurants ────────────────────────────────────────
# If any restaurants failed LLM generation, fill them with a template-based menu
# so the simulation step has complete coverage

FALLBACK_TEMPLATES = {
    'biryani': {
        'items': [
            {'name':'Chicken Dum Biryani','category':'main','cuisine_tag':'biryani','price':280,'is_veg':False,'calories':650,'popularity_score':0.9},
            {'name':'Veg Biryani','category':'main','cuisine_tag':'biryani','price':200,'is_veg':True,'calories':520,'popularity_score':0.75},
            {'name':'Raita','category':'side','cuisine_tag':'biryani','price':60,'is_veg':True,'calories':80,'popularity_score':0.85},
            {'name':'Salan','category':'side','cuisine_tag':'biryani','price':70,'is_veg':False,'calories':120,'popularity_score':0.7},
            {'name':'Shorba','category':'drink','cuisine_tag':'biryani','price':55,'is_veg':False,'calories':60,'popularity_score':0.6},
            {'name':'Gulab Jamun','category':'dessert','cuisine_tag':'north_indian','price':70,'is_veg':True,'calories':250,'popularity_score':0.8},
            {'name':'Cold Drink','category':'drink','cuisine_tag':'beverages','price':50,'is_veg':True,'calories':140,'popularity_score':0.7},
        ],
        'combo_rules': [
            {'if_category':'main','if_cuisine_tag':'biryani','then_category':'side','then_cuisine_tag':'biryani','weight':0.75},
            {'if_category':'main','if_cuisine_tag':'biryani','then_category':'drink','then_cuisine_tag':'biryani','weight':0.4},
            {'if_category':'side','if_cuisine_tag':'biryani','then_category':'dessert','then_cuisine_tag':'north_indian','weight':0.5},
        ]
    }
    # (abbreviated — the code below fills all cuisine types)
}

def make_fallback_menu(restaurant):
    cuisine = restaurant['cuisine']
    tmpl = FALLBACK_TEMPLATES.get(cuisine, FALLBACK_TEMPLATES['biryani'])
    # scale prices to restaurant's tier
    mult = PRICE_TIERS[restaurant['price_tier']]
    items = []
    for it in tmpl['items']:
        item = it.copy()
        item['price'] = int(it['price'] * mult)
        items.append(item)
    return {'items': items, 'combo_rules': tmpl['combo_rules']}

for rid in failed:
    rest = restaurants_df[restaurants_df['restaurant_id']==rid].iloc[0]
    generated_menus[rid] = make_fallback_menu(rest.to_dict())
    print(f'  Fallback applied for {rid}')

print(f'All {len(restaurants_df)} restaurants now have menus.')

## 📋 Step 4 — Inspect LLM output quality

In [ ]:
# Quick quality check — look at a few generated menus
sample_ids = list(generated_menus.keys())[:3]
for rid in sample_ids:
    rest = restaurants_df[restaurants_df['restaurant_id']==rid].iloc[0]
    menu_data = generated_menus[rid]
    items = menu_data['items']
    combos = menu_data.get('combo_rules', [])
    print(f"\n{'='*60}")
    print(f"Restaurant: {rest['name']} ({rest['cuisine']}, {rest['price_tier']})")
    print(f"Personality: {rest['personality']}")
    print(f"Items: {len(items)} | Combo rules: {len(combos)}")
    print(f"\nSample items:")
    for it in items[:6]:
        veg = '🟢' if it.get('is_veg') else '🔴'
        print(f"  {veg} [{it['category']:7}] {it['name']:35} ₹{it['price']}")
    print(f"\nCombo rules:")
    for rule in combos[:4]:
        print(f"  if {rule['if_category']}({rule['if_cuisine_tag']}) → suggest {rule['then_category']}({rule['then_cuisine_tag']}) [w={rule['weight']}]")

# Stats
all_items_flat = [it for m in generated_menus.values() for it in m['items']]
unique_names = len(set(it['name'] for it in all_items_flat))
print(f"\n{'='*60}")
print(f"Total item instances : {len(all_items_flat)}")
print(f"Unique dish names    : {unique_names}")
print(f"Avg items/restaurant : {len(all_items_flat)/len(generated_menus):.1f}")

## 🗃️ Step 5 — Build structured menu_items table

In [ ]:

# ═══════════════════════════════════════════════════════
# CELL: build-menu-table
# DEPENDS ON: run-generation (generated_menus), restaurant-profiles
# PRODUCES: menu_df, restaurant_menus
# ═══════════════════════════════════════════════════════

# --- REVENUE FEATURE 1: margin_score per category ---
CATEGORY_BASE_MARGIN = {
    'drink':   0.65,
    'dessert': 0.55,
    'snack':   0.50,
    'side':    0.40,
    'main':    0.30,
}

menu_rows = []
restaurant_menus = {}   # restaurant_id -> list of item dicts (with item_id added)
item_counter = 0

for _, rest in restaurants_df.iterrows():
    rid = rest['restaurant_id']
    menu_data = generated_menus.get(rid, {})
    raw_items = menu_data.get('items', [])

    menu = []
    for raw in raw_items:
        try:
            cat = str(raw.get('category','main')).lower()
            if cat not in ['main','side','drink','dessert','snack']:
                cat = 'main'
            item = {
                'item_id':          f'I{item_counter:06d}',
                'restaurant_id':    rid,
                'name':             str(raw.get('name', f'Item_{item_counter}')),
                'category':         cat,
                'cuisine_tag':      str(raw.get('cuisine_tag', rest['cuisine'])).lower(),
                'price':            max(30, int(float(raw.get('price', 150)))),
                'is_veg':           bool(raw.get('is_veg', True)),
                'calories':         max(50, int(float(raw.get('calories', 300)))),
                'popularity_score': round(min(1.0, max(0.1, float(raw.get('popularity_score', 0.6)))), 2),
                'margin_score':     round(
                    CATEGORY_BASE_MARGIN.get(cat, 0.35) * np.random.uniform(0.85, 1.15), 3
                ),
            }
            menu.append(item)
            menu_rows.append(item)
            item_counter += 1
        except Exception as e:
            print(f'  [SKIP] Bad item in {rid}: {e}')
            continue

    restaurant_menus[rid] = menu

menu_df = pd.DataFrame(menu_rows)
print(f'menu_items table: {len(menu_df)} rows, {menu_df["name"].nunique()} unique dish names')
print(menu_df.groupby('category').size().sort_values(ascending=False))
print(f'margin_score mean by category:\n{menu_df.groupby("category")["margin_score"].mean().round(3)}')


## 👥 Step 6 — Generate users

In [ ]:

# ═══════════════════════════════════════════════════════
# CELL: gen-users
# DEPENDS ON: imports, restaurant-profiles
# PRODUCES: users_df
# ═══════════════════════════════════════════════════════

N_USERS = 2000
USER_SEGMENTS = ['budget','mid','premium','late_night_binger','health_conscious','family']

SLOT_HOURS = {
    'breakfast':  list(range(6, 11)),
    'lunch':      list(range(11, 15)),
    'snack':      list(range(15, 18)),
    'dinner':     list(range(18, 23)),
    'late_night': list(range(23, 24)) + list(range(0, 4)),
}

user_rows = []
for i in range(N_USERS):
    seg = random.choice(USER_SEGMENTS)
    preferred = random.sample(list(CUISINE_PROFILES.keys()), k=random.randint(2,3))
    order_freq = {'budget':random.randint(2,6), 'mid':random.randint(4,12),
                  'premium':random.randint(6,20), 'late_night_binger':random.randint(5,15),
                  'health_conscious':random.randint(3,8), 'family':random.randint(4,10)}[seg]
    avg_spend  = {'budget':random.randint(150,350), 'mid':random.randint(300,600),
                  'premium':random.randint(500,1200), 'late_night_binger':random.randint(250,500),
                  'health_conscious':random.randint(250,500), 'family':random.randint(400,900)}[seg]

    # Preferred slots per segment
    if seg == 'late_night_binger':
        pref_slots = random.sample(['late_night', 'dinner'], k=random.randint(1,2))
    elif seg == 'health_conscious':
        pref_slots = random.sample(['breakfast', 'lunch'], k=random.randint(1,2))
    elif seg == 'family':
        pref_slots = random.sample(['lunch', 'dinner'], k=random.randint(1,2))
    else:
        pref_slots = random.sample(list(SLOT_HOURS.keys()), k=random.randint(2,3))

    # Behavioural rates
    addon_rate   = round(random.uniform(0.1, 0.8), 3)
    drink_rate   = round(random.uniform(0.05, 0.6), 3)
    dessert_rate = round(random.uniform(0.05, 0.5), 3)

    user_rows.append({
        'user_id':            f'U{i:05d}',
        'city':               random.choice(CITIES),
        'segment':            seg,
        'preferred_cuisines': json.dumps(preferred),
        'orders_per_month':   order_freq,
        'avg_order_value':    avg_spend,
        'is_veg':             random.random() < 0.38,
        'account_age_days':   random.randint(30, 1200),
        'lifetime_orders':    random.randint(order_freq, order_freq * 24),
        'addon_rate':         addon_rate,
        'drink_rate':         drink_rate,
        'dessert_rate':       dessert_rate,
        'price_sensitivity':  round(random.uniform(0.2, 1.0), 3),
        'cuisine_loyalty':    round(random.uniform(0.1, 1.0), 3),
        'avg_items_per_order':round(random.uniform(1.5, 4.5), 2),
        'preferred_slots':    json.dumps(pref_slots),
        'is_cold_start':      random.random() < 0.15,
    })

users_df = pd.DataFrame(user_rows)
print(f'users: {len(users_df)} rows')
print(users_df['segment'].value_counts())
print(f'Cold start users: {users_df["is_cold_start"].sum()} ({users_df["is_cold_start"].mean()*100:.1f}%)')


## 📦 Step 7 — Generate historical orders

In [ ]:

# ═══════════════════════════════════════════════════════
# CELL: gen-orders
# DEPENDS ON: gen-users, build-menu-table, restaurant-profiles
# PRODUCES: orders_df, user_item_history, item_freq, pair_freq, n_baskets
# ═══════════════════════════════════════════════════════

from collections import defaultdict

N_ORDERS = 40_000

# --- IMPROVEMENT 2: Temporal concentration of orders ---
sim_end = datetime(2025, 1, 1)

def sample_order_date(sim_end):
    if random.random() < 0.6:
        days_back = random.randint(0, 90)
    else:
        days_back = random.randint(91, 364)
    return sim_end - timedelta(days=days_back, hours=random.randint(0, 23))

def hour_dist():
    w = np.array([0.5,0.3,0.3,0.2,0.2,0.3, 0.8,1.5,2.0,1.5,1.2,2.5,
                  4.0,4.5,3.5,2.0,1.8,2.0, 3.0,4.0,4.5,3.5,2.0,1.0], dtype=float)
    return w / w.sum()

def user_hour_dist(pref_slots):
    w = np.array([0.5,0.3,0.3,0.2,0.2,0.3, 0.8,1.5,2.0,1.5,1.2,2.5,
                  4.0,4.5,3.5,2.0,1.8,2.0, 3.0,4.0,4.5,3.5,2.0,1.0], dtype=float)
    boost_hours = []
    for slot in pref_slots:
        boost_hours.extend(SLOT_HOURS.get(slot, []))
    for h in boost_hours:
        w[h % 24] *= 2.5
    return w / w.sum()

def get_meal_slot(hour):
    if 6  <= hour < 11: return 'breakfast'
    if 11 <= hour < 15: return 'lunch'
    if 15 <= hour < 18: return 'snack'
    if 18 <= hour < 23: return 'dinner'
    return 'late_night'

def pick_restaurant(uinfo):
    pref = json.loads(uinfo['preferred_cuisines'])
    candidates = restaurants_df[restaurants_df['cuisine'].isin(pref)]['restaurant_id'].tolist()
    if not candidates:
        candidates = rest_ids
    return random.choice(candidates)

user_ids  = users_df['user_id'].tolist()
rest_ids  = restaurants_df['restaurant_id'].tolist()
user_lkp  = users_df.set_index('user_id').to_dict('index')

# --- IMPROVEMENT 1: Power-law order distribution ---
user_order_weights = np.array([
    user_lkp[uid]['orders_per_month'] ** 1.5
    for uid in user_ids
], dtype=float)
user_order_weights /= user_order_weights.sum()

# --- BUG 3: Restaurant repeat behavior ---
user_last_restaurant = {}
user_restaurant_counts = defaultdict(lambda: defaultdict(int))

order_rows = []
for i in range(N_ORDERS):
    uid   = np.random.choice(user_ids, p=user_order_weights)
    uinfo = user_lkp[uid]

    # BUG 3: loyalty-based restaurant repeat
    r = random.random()
    if uid in user_last_restaurant and r < 0.4:
        rid = user_last_restaurant[uid]
    elif user_restaurant_counts[uid] and r < 0.65:
        top3 = sorted(user_restaurant_counts[uid].items(),
                      key=lambda x: x[1], reverse=True)[:3]
        rid = random.choice(top3)[0]
    else:
        rid = pick_restaurant(uinfo)

    user_last_restaurant[uid] = rid
    user_restaurant_counts[uid][rid] += 1

    menu = restaurant_menus.get(rid, [])
    if not menu: continue

    pref_slots = json.loads(uinfo['preferred_slots'])
    hour  = int(np.random.choice(range(24), p=user_hour_dist(pref_slots)))
    odate = sample_order_date(sim_end)
    meal_slot = get_meal_slot(hour)

    mains  = [it for it in menu if it['category'] == 'main']

    # IMPROVEMENT 3: Veg filter in gen-orders
    eligible = list(menu)
    if uinfo['is_veg']:
        veg_eligible = [it for it in eligible if it['is_veg']]
        eligible = veg_eligible if len(veg_eligible) >= 2 else eligible

    mains_eligible = [it for it in eligible if it['category'] == 'main']
    others_eligible = [it for it in eligible if it['category'] != 'main']

    picked = [random.choice(mains_eligible)] if mains_eligible else []
    extra  = min(np.random.choice([1,2,3], p=[0.35,0.40,0.25]), len(others_eligible))
    if extra > 0:
        picked += random.sample(others_eligible, extra)

    has_addon = int(len(picked) > 1)
    item_categories = json.dumps([it['category'] for it in picked])

    order_rows.append({
        'order_id':        f'O{i:07d}',
        'user_id':         uid,
        'restaurant_id':   rid,
        'order_datetime':  odate.strftime('%Y-%m-%d %H:%M:%S'),
        'meal_slot':       meal_slot,
        'items':           json.dumps([it['item_id'] for it in picked]),
        'item_names':      json.dumps([it['name'] for it in picked]),
        'item_categories': item_categories,
        'n_items':         len(picked),
        'total_value':     sum(it['price'] for it in picked),
        'has_addon':       has_addon,
    })

orders_df = pd.DataFrame(order_rows)
print(f'orders: {len(orders_df)} rows | avg order value: ₹{orders_df["total_value"].mean():.0f}')
print(f'has_addon rate: {orders_df["has_addon"].mean()*100:.1f}%')
print(f'date range: {orders_df["order_datetime"].min()} → {orders_df["order_datetime"].max()}')


In [ ]:

# ═══════════════════════════════════════════════════════
# CELL: gen-features
# DEPENDS ON: gen-users, gen-orders, build-menu-table
# PRODUCES: user_feat, rest_feat (merged into user_lkp / rest_lookup for gen-sessions)
# ═══════════════════════════════════════════════════════

# --- Parse order_datetime ---
orders_df['order_dt'] = pd.to_datetime(orders_df['order_datetime'])

# --- User aggregate features ---
now = pd.Timestamp(sim_end)

user_agg = orders_df.groupby('user_id').agg(
    user_total_orders    = ('order_id',   'count'),
    user_orders_30d      = ('order_dt',   lambda x: ((now - x).dt.days <= 30).sum()),
    user_orders_7d       = ('order_dt',   lambda x: ((now - x).dt.days <= 7).sum()),
    user_days_since_last = ('order_dt',   lambda x: (now - x.max()).days),
    user_median_item_price_90d = ('total_value', 'median'),
    user_addon_rate_30d  = ('has_addon',  'mean'),
).reset_index()

user_agg.rename(columns={'user_days_since_last': 'user_days_since_last_order'}, inplace=True)

# Drink/dessert rate per user
drink_rows, dessert_rows = [], []
for _, row in orders_df.iterrows():
    cats = json.loads(row['item_categories'])
    drink_rows.append({'user_id': row['user_id'], 'has_drink': int('drink' in cats)})
    dessert_rows.append({'user_id': row['user_id'], 'has_dessert': int('dessert' in cats)})

drink_rate_df   = pd.DataFrame(drink_rows).groupby('user_id')['has_drink'].mean().reset_index().rename(columns={'has_drink':'user_drink_rate_30d'})
dessert_rate_df = pd.DataFrame(dessert_rows).groupby('user_id')['has_dessert'].mean().reset_index().rename(columns={'has_dessert':'user_dessert_rate_30d'})

user_feat = users_df[['user_id','segment','is_veg','is_cold_start',
                       'addon_rate','drink_rate','dessert_rate',
                       'price_sensitivity','cuisine_loyalty','avg_items_per_order',
                       'preferred_slots']].copy()
user_feat = user_feat.merge(user_agg, on='user_id', how='left')
user_feat = user_feat.merge(drink_rate_df,   on='user_id', how='left')
user_feat = user_feat.merge(dessert_rate_df, on='user_id', how='left')

user_feat['user_addon_rate_30d'].fillna(0, inplace=True)
user_feat['user_drink_rate_30d'].fillna(0, inplace=True)
user_feat['user_dessert_rate_30d'].fillna(0, inplace=True)
user_feat['user_days_since_last_order'].fillna(999, inplace=True)
user_feat['user_orders_30d'].fillna(0, inplace=True)
user_feat['user_orders_7d'].fillna(0, inplace=True)
user_feat['user_median_item_price_90d'].fillna(200, inplace=True)

# --- BUG 5 fix — zero out cold start users ---
cold_ids = users_df[users_df['is_cold_start']]['user_id'].tolist()
for col in ['user_days_since_last_order','user_orders_30d','user_orders_7d',
            'user_addon_rate_30d','user_drink_rate_30d','user_dessert_rate_30d',
            'user_median_item_price_90d']:
    user_feat.loc[user_feat['user_id'].isin(cold_ids), col] = np.nan

user_feat.fillna({
    'user_days_since_last_order':  999,
    'user_orders_30d':             0,
    'user_orders_7d':              0,
    'user_addon_rate_30d':         -1,
    'user_drink_rate_30d':         -1,
    'user_dessert_rate_30d':       -1,
    'user_median_item_price_90d':  -1,
}, inplace=True)

# --- IMPROVEMENT 4: slot addon rate ---
slot_addon = orders_df.groupby(['user_id','meal_slot'])['has_addon'].mean().reset_index()
slot_addon_wide = slot_addon.pivot(
    index='user_id', columns='meal_slot', values='has_addon'
).reset_index()
slot_addon_wide.columns = ['user_id'] + [
    f'user_addon_rate_{c}' for c in slot_addon_wide.columns[1:]
]
slot_addon_wide.fillna(-1, inplace=True)
user_feat = user_feat.merge(slot_addon_wide, on='user_id', how='left')

# --- BUG 4 fix: lift + pair_seen_before (replaces pmi) ---
item_freq = defaultdict(int)
pair_freq = defaultdict(int)

for _, row in tqdm(orders_df.iterrows(), total=len(orders_df), desc='Computing lift/pair stats'):
    iids = json.loads(row['items'])
    for iid in iids:
        item_freq[iid] += 1
    for j in range(len(iids)):
        for k in range(j+1, len(iids)):
            pair = tuple(sorted([iids[j], iids[k]]))
            pair_freq[pair] += 1

n_baskets = len(orders_df)

def lift(item_a, item_b, n_baskets):
    pair = tuple(sorted([item_a, item_b]))
    p_ab = pair_freq.get(pair, 0) / max(n_baskets, 1)
    p_a  = item_freq.get(item_a, 1) / max(n_baskets, 1)
    p_b  = item_freq.get(item_b, 1) / max(n_baskets, 1)
    if p_a * p_b == 0:
        return 1.0
    return p_ab / (p_a * p_b)

def pair_seen_before(item_a, item_b):
    pair = tuple(sorted([item_a, item_b]))
    return int(pair_freq.get(pair, 0) > 0)

# --- REVENUE FEATURE 3: user_price_upgrade_tendency ---
upgrade_rows = []
for _, row in orders_df.iterrows():
    iids = json.loads(row['items'])
    cats = json.loads(row['item_categories'])
    if len(iids) < 2:
        continue
    prices = []
    for iid in iids:
        item = menu_df[menu_df['item_id'] == iid]
        if not item.empty:
            prices.append(item.iloc[0]['price'])
    if len(prices) >= 2:
        main_price = prices[0]
        addon_avg  = np.mean(prices[1:])
        upgrade_rows.append({
            'user_id': row['user_id'],
            'upgrade_delta': addon_avg - main_price
        })

if upgrade_rows:
    upgrade_df = pd.DataFrame(upgrade_rows)
    user_upgrade = upgrade_df.groupby('user_id')['upgrade_delta'].mean().reset_index(
        ).rename(columns={'upgrade_delta': 'user_price_upgrade_tendency'})
    user_feat = user_feat.merge(user_upgrade, on='user_id', how='left')
user_feat['user_price_upgrade_tendency'] = user_feat.get('user_price_upgrade_tendency', pd.Series(0, index=user_feat.index))
user_feat['user_price_upgrade_tendency'].fillna(0, inplace=True)

# --- REVENUE FEATURE 6: rest_avg_margin ---
rest_avg_margin = menu_df.groupby('restaurant_id')['margin_score'].mean().reset_index(
    ).rename(columns={'margin_score': 'rest_avg_margin'})
rest_feat = restaurants_df[['restaurant_id','name','city','cuisine','price_tier','rating','monthly_orders']].copy()
rest_feat = rest_feat.merge(rest_avg_margin, on='restaurant_id', how='left')
rest_feat['rest_avg_margin'].fillna(0.4, inplace=True)

print(f'user_feat: {len(user_feat)} rows × {len(user_feat.columns)} cols')
print(f'rest_feat: {len(rest_feat)} rows × {len(rest_feat.columns)} cols')
print(f'item_freq: {len(item_freq)} items | pair_freq: {len(pair_freq)} pairs | n_baskets: {n_baskets}')
print(f'user_price_upgrade_tendency mean: {user_feat["user_price_upgrade_tendency"].mean():.2f}')


## 🛒 Step 8 — Simulate cart sessions (main training table)

In [ ]:

# ═══════════════════════════════════════════════════════
# CELL: gen-sessions
# DEPENDS ON: gen-users, gen-orders, gen-features, build-menu-table
# PRODUCES: cart_df  (assembled from parquet chunks — OOM-safe at 80K sessions)
# ═══════════════════════════════════════════════════════

import pyarrow as pa
import pyarrow.parquet as pq

N_SESSIONS   = 80_000
CHUNK_SIZE   = 5_000          # flush to parquet every this many sessions
CHUNK_DIR    = '/content/session_chunks'
os.makedirs(CHUNK_DIR, exist_ok=True)

# ── Weather proxy lookup ─────────────────────────────────────────────────────
# Synthetic monthly mean temperature (°C) per city — grounded in real climate normals
CITY_TEMP = {
    # city: [Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, Oct, Nov, Dec]
    'Mumbai':    [24,25,27,29,31,30,28,28,28,29,27,25],
    'Delhi':     [14,17,23,30,36,38,34,33,31,26,19,14],
    'Bengaluru': [21,23,25,27,27,24,23,23,23,23,21,20],
    'Hyderabad': [22,25,28,32,35,31,28,28,28,27,23,21],
    'Pune':      [20,22,25,29,31,28,25,24,24,26,22,19],
    'Chennai':   [25,26,28,31,34,33,31,31,30,28,25,24],
    'Kolkata':   [18,21,27,31,33,32,30,30,30,28,23,18],
}
DEFAULT_TEMP = [25,26,28,30,32,30,28,28,28,27,25,24]

def get_weather_temp(city: str, month: int) -> float:
    """Return synthetic temp for city/month with small Gaussian noise."""
    temps = CITY_TEMP.get(city, DEFAULT_TEMP)
    base  = temps[month - 1]
    return round(base + np.random.normal(0, 1.5), 1)

# ── Pre-index lookups ────────────────────────────────────────────────────────
user_feat_lkp = user_feat.set_index('user_id').to_dict('index')
for uid in user_feat_lkp:
    urow = users_df[users_df['user_id'] == uid]
    if not urow.empty:
        urow = urow.iloc[0]
        user_feat_lkp[uid]['addon_rate']         = urow['addon_rate']
        user_feat_lkp[uid]['drink_rate']          = urow['drink_rate']
        user_feat_lkp[uid]['dessert_rate']        = urow['dessert_rate']
        user_feat_lkp[uid]['is_veg']              = urow['is_veg']
        user_feat_lkp[uid]['preferred_cuisines']  = urow['preferred_cuisines']
        user_feat_lkp[uid]['avg_order_value']     = urow['avg_order_value']
        user_feat_lkp[uid]['orders_per_month']    = urow['orders_per_month']

user_lookup = user_feat_lkp
rest_lookup = rest_feat.set_index('restaurant_id').to_dict('index')

rest_combo_rules = {rid: md.get('combo_rules', [])
                    for rid, md in generated_menus.items()}

user_item_history = {}
for _, row in orders_df.iterrows():
    uid = row['user_id']
    for iid in json.loads(row['items']):
        user_item_history[(uid, iid)] = user_item_history.get((uid, iid), 0) + 1

item_margin_lookup = menu_df.set_index('item_id')['margin_score'].to_dict()

# ── Constants ────────────────────────────────────────────────────────────────
SLOT_BOOST = {
    'breakfast':  {'snack':1.3, 'drink':1.4},
    'lunch':      {'main':1.3, 'side':1.2},
    'snack':      {'snack':1.5, 'drink':1.3},
    'dinner':     {'main':1.2, 'dessert':1.3},
    'late_night': {'main':1.1, 'drink':1.5, 'dessert':1.4},
}
MEAL_SLOTS_NEEDED = ['main', 'side', 'drink']

# Temporal split boundaries
TRAIN_END = datetime(2024, 10, 1)
VAL_END   = datetime(2024, 11, 15)

def temporal_split(dt: datetime) -> str:
    if dt < TRAIN_END:  return 'train'
    if dt < VAL_END:    return 'val'
    return 'test'

def combo_prob(cart_items, candidate, meal_slot, uinfo, combo_rules):
    if uinfo['is_veg'] and not candidate['is_veg']:
        return 0.0
    cand_cat  = candidate['category']
    cand_ctag = candidate['cuisine_tag']
    if not cart_items:
        base = 0.6 if cand_cat == 'main' else 0.15
    else:
        cart_cats  = [it['category'] for it in cart_items]
        cart_ctags = [it['cuisine_tag'] for it in cart_items]
        base = candidate['popularity_score'] * 0.3
        for rule in combo_rules:
            if (rule['if_category'] in cart_cats and
                rule['if_cuisine_tag'] in cart_ctags and
                rule['then_category'] == cand_cat and
                rule['then_cuisine_tag'] == cand_ctag):
                base += float(rule.get('weight', 0.5))
        cnt = cart_cats.count(cand_cat)
        base *= (0.3 if cnt >= 2 else 0.6 if cnt == 1 else 1.0)
        base *= SLOT_BOOST.get(meal_slot, {}).get(cand_cat, 1.0)
        if not uinfo.get('is_cold_start', False):
            if cand_cat != 'main':    base *= (0.5 + uinfo['addon_rate'])
            if cand_cat == 'drink':   base *= (0.5 + uinfo['drink_rate'])
            if cand_cat == 'dessert': base *= (0.5 + uinfo['dessert_rate'])
        seg = uinfo.get('segment', '')
        if seg == 'health_conscious' and cand_cat == 'dessert': base *= 0.4
        if seg == 'late_night_binger' and cand_cat == 'drink':  base *= 1.5
        if seg == 'budget'  and candidate['price'] > 250:       base *= 0.5
        if seg == 'premium' and candidate['price'] < 80:        base *= 0.7
    return min(float(base), 1.0)

user_ids = users_df['user_id'].tolist()
rest_ids = restaurants_df['restaurant_id'].tolist()

user_order_weights_sess = np.array(
    [user_lookup[uid]['orders_per_month'] ** 1.5 for uid in user_ids], dtype=float)
user_order_weights_sess /= user_order_weights_sess.sum()

# TWO TRAINING OBJECTIVES:
# 1. added (binary)                 → CTR / acceptance rate
# 2. revenue_weighted_label (float) → AOV / revenue
# Hybrid: 0.6 * added + 0.4 * (revenue_weighted_label / revenue_weighted_label.max())

# ── Main session loop — chunked writes ───────────────────────────────────────
chunk_rows  = []
chunk_files = []
total_rows  = 0
chunk_idx   = 0

for sid in tqdm(range(N_SESSIONS), desc='Simulating cart sessions'):
    uid   = np.random.choice(user_ids, p=user_order_weights_sess)
    uinfo = user_lookup[uid]
    rid   = pick_restaurant(uinfo)

    menu  = restaurant_menus.get(rid, [])
    if len(menu) < 3:
        continue

    rinfo       = rest_lookup.get(rid, {})
    combo_rules = rest_combo_rules.get(rid, [])

    pref_slots = json.loads(uinfo['preferred_slots'])
    hour       = int(np.random.choice(range(24), p=user_hour_dist(pref_slots)))
    meal_slot  = get_meal_slot(hour)

    # Synthetic session date for temporal split
    session_date = sample_order_date(sim_end)
    split        = temporal_split(session_date)
    month        = session_date.month

    # Weather proxy
    city          = rinfo.get('city', 'Mumbai')
    weather_temp  = get_weather_temp(city, month)
    is_hot        = int(weather_temp >= 30)
    weather_drink = round(weather_temp / 40.0, 3)

    order_size = np.random.choice([1,2,3,4,5], p=[0.10,0.35,0.35,0.15,0.05])
    n_steps    = order_size - 1
    if n_steps == 0:
        continue

    if uinfo['is_veg']:
        veg_menu      = [it for it in menu if it['is_veg']]
        eligible_menu = veg_menu if len(veg_menu) >= 3 else menu
    else:
        eligible_menu = menu

    cart = []

    for step in range(n_steps):
        remaining = [it for it in eligible_menu if it not in cart]
        if not remaining:
            break

        probs = np.array([combo_prob(cart, it, meal_slot, uinfo, combo_rules)
                          for it in remaining])
        probs += np.random.uniform(0, 0.05, size=len(probs))
        probs  = np.clip(probs, 0.01, 1.0)
        probs /= probs.sum()

        chosen_idx = np.random.choice(len(remaining), p=probs)
        chosen     = remaining[chosen_idx]

        cart_ids   = [it['item_id'] for it in cart]
        cart_cats  = [it['category'] for it in cart]
        cart_total = sum(it['price'] for it in cart)
        cart_avg   = cart_total / len(cart) if cart else 0
        cat_flags  = {c: int(c in cart_cats)
                      for c in ['main','side','drink','dessert','snack']}
        cart_momentum = (sum(1 for s in MEAL_SLOTS_NEEDED if s in cart_cats)
                         / len(MEAL_SLOTS_NEEDED))

        # ── FULL NEGATIVES: every remaining item that wasn't chosen ──────────
        candidates_with_labels = (
            [(chosen, 1)] +
            [(it, 0) for it in remaining if it is not chosen]
        )

        pref_c = json.loads(uinfo['preferred_cuisines'])
        slot_col = f'user_addon_rate_{meal_slot}'

        for candidate, label in candidates_with_labels:
            prev      = user_item_history.get((uid, candidate['item_id']), 0)
            affinity  = int(candidate['cuisine_tag'] in pref_c
                            or rinfo.get('cuisine','') in pref_c)
            margin    = item_margin_lookup.get(candidate['item_id'], 0.4)

            slot_urgency = (min(1.0, len(cart) / 3.0)
                            if candidate['category'] not in cart_cats else 0.0)

            candidate_revenue_potential = round(
                (candidate['price'] / max(cart_avg, 50)) * margin, 4)

            revenue_weighted_label = round(
                label * candidate['price'] * margin / 100.0, 4)

            aov_lift_if_added = round(candidate['price'] / max(cart_total, 50), 4)

            price_anchor  = cart_avg if cart_avg > 0 else uinfo.get(
                'user_median_item_price_90d', 200)
            candidate_in_price_sweet_spot = int(
                price_anchor * 0.20 <= candidate['price'] <= price_anchor * 0.60)

            user_addon_rate_this_slot = round(
                float(uinfo.get(slot_col, -1)), 3)

            chunk_rows.append({
                'session_id':                    f'S{sid:07d}',
                'step':                          step,
                'split':                         split,
                'session_date':                  session_date.strftime('%Y-%m-%d'),
                'user_id':                       uid,
                'restaurant_id':                 rid,
                'city':                          city,
                'hour':                          hour,
                'meal_slot':                     meal_slot,
                'cart_item_ids':                 json.dumps(cart_ids),
                'cart_categories':               json.dumps(cart_cats),
                'cart_total':                    cart_total,
                'cart_size':                     len(cart),
                'cart_momentum':                 round(cart_momentum, 4),
                'has_main':                      cat_flags['main'],
                'has_side':                      cat_flags['side'],
                'has_drink':                     cat_flags['drink'],
                'has_dessert':                   cat_flags['dessert'],
                'has_snack':                     cat_flags['snack'],
                'candidate_item_id':             candidate['item_id'],
                'candidate_name':                candidate['name'],
                'candidate_category':            candidate['category'],
                'candidate_cuisine_tag':         candidate['cuisine_tag'],
                'candidate_price':               candidate['price'],
                'candidate_is_veg':              candidate['is_veg'],
                'candidate_calories':            candidate['calories'],
                'candidate_popularity':          candidate['popularity_score'],
                'candidate_margin_score':        round(margin, 4),
                'candidate_revenue_potential':   candidate_revenue_potential,
                'candidate_in_price_sweet_spot': candidate_in_price_sweet_spot,
                'candidate_slot_urgency':        round(slot_urgency, 4),
                'price_delta_cart_avg':          round(candidate['price'] - cart_avg, 2),
                'pair_max_lift':                 round(max(
                    (lift(candidate['item_id'], cit, n_baskets) for cit in cart_ids),
                    default=1.0), 4),
                'pair_seen_before_flag':         int(any(
                    pair_seen_before(candidate['item_id'], cit) for cit in cart_ids)),
                'weather_temp_c':                weather_temp,
                'is_hot_weather':                is_hot,
                'weather_drink_affinity':        weather_drink,
                'user_segment':                  uinfo.get('segment',''),
                'user_avg_order_value':          uinfo.get('avg_order_value', 0),
                'user_is_veg':                   uinfo['is_veg'],
                'user_orders_per_month':         uinfo.get('orders_per_month', 0),
                'user_ordered_before':           min(prev, 5),
                'user_cuisine_affinity':         affinity,
                'user_addon_rate_30d':           round(min(1.0, max(0.0,
                    uinfo.get('user_addon_rate_30d', 0.3) + np.random.normal(0, 0.05))), 3),
                'user_drink_rate_30d':           round(min(1.0, max(0.0,
                    uinfo.get('user_drink_rate_30d', 0.2) + np.random.normal(0, 0.04))), 3),
                'user_dessert_rate_30d':         round(min(1.0, max(0.0,
                    uinfo.get('user_dessert_rate_30d', 0.15) + np.random.normal(0, 0.04))), 3),
                'user_addon_rate_this_slot':     user_addon_rate_this_slot,
                'user_price_upgrade_tendency':   round(float(uinfo.get(
                    'user_price_upgrade_tendency', 0)), 4),
                'user_days_since_last_order':    uinfo.get('user_days_since_last_order', 999),
                'user_orders_30d':               uinfo.get('user_orders_30d', 0),
                'user_orders_7d':                uinfo.get('user_orders_7d', 0),
                'user_median_item_price_90d':    uinfo.get('user_median_item_price_90d', -1),
                'is_cold_start':                 int(uinfo.get('is_cold_start', False)),
                'rest_cuisine':                  rinfo.get('cuisine',''),
                'rest_price_tier':               rinfo.get('price_tier',''),
                'rest_rating':                   rinfo.get('rating', 0),
                'rest_avg_margin':               round(float(rinfo.get('rest_avg_margin', 0.4)), 4),
                'revenue_weighted_label':        revenue_weighted_label,
                'aov_lift_if_added':             aov_lift_if_added,
                'added':                         label,
            })

        cart.append(chosen)

    # ── Flush chunk to parquet every CHUNK_SIZE sessions ─────────────────────
    if (sid + 1) % CHUNK_SIZE == 0 and chunk_rows:
        chunk_path = f'{CHUNK_DIR}/chunk_{chunk_idx:04d}.parquet'
        pd.DataFrame(chunk_rows).to_parquet(chunk_path, index=False)
        total_rows += len(chunk_rows)
        chunk_files.append(chunk_path)
        chunk_rows  = []
        chunk_idx  += 1

    # Running stats every 5000 sessions
    if sid % 5000 == 0 and sid > 0:
        print(f"  [{sid}/{N_SESSIONS}] chunks written: {chunk_idx} | "
              f"total rows so far: {total_rows:,}")

# Flush any remaining rows
if chunk_rows:
    chunk_path = f'{CHUNK_DIR}/chunk_{chunk_idx:04d}.parquet'
    pd.DataFrame(chunk_rows).to_parquet(chunk_path, index=False)
    total_rows += len(chunk_rows)
    chunk_files.append(chunk_path)

# ── Assemble final DataFrame ─────────────────────────────────────────────────
print(f'\nAssembling {len(chunk_files)} chunks ({total_rows:,} rows)...')
cart_df = pd.concat([pd.read_parquet(f) for f in chunk_files], ignore_index=True)

print(f'cart_sessions: {len(cart_df):,} rows | '
      f'positive rate: {cart_df["added"].mean()*100:.2f}% | '
      f'columns: {len(cart_df.columns)}')
print(f'\nTemporal split distribution:')
print(cart_df['split'].value_counts().sort_index().to_string())
print(f'\nWeather feature sample — mean temp by city:')
print(cart_df.groupby('city')['weather_temp_c'].mean().round(1).to_string())


In [ ]:

# ═══════════════════════════════════════════════════════
# CELL: validate
# INPUT: cart_df, users_df, menu_df, orders_df
# OUTPUT: printed quality report, no new variables
# ═══════════════════════════════════════════════════════

print("=" * 65)
print("DATASET QUALITY REPORT")
print("=" * 65)

# 1. Positive rate
pos_rate = cart_df['added'].mean()
status = "OK" if 0.01 <= pos_rate <= 0.10 else "WARNING"
print(f"[{status}] Positive rate: {pos_rate*100:.2f}%  "
      f"(full negatives → expect lower than 10%)")

# 2. Veg constraint
veg_violations = cart_df[
    (cart_df['user_is_veg'] == True) &
    (cart_df['candidate_is_veg'] == False) &
    (cart_df['added'] == 1)
]
status = "OK" if len(veg_violations) == 0 else "WARNING"
print(f"[{status}] Veg violations: {len(veg_violations)} (expected: 0)")

# 3. Cold start coverage
cold_pct = cart_df['is_cold_start'].mean() * 100
status = "OK" if 10 <= cold_pct <= 20 else "WARNING"
print(f"[{status}] Cold start sessions: {cold_pct:.1f}% (expected: ~15%)")

# 4. Null rates
null_rates = cart_df.isnull().mean()
high_null = null_rates[null_rates > 0.3].index.tolist()
status = "OK" if not high_null else "WARNING"
print(f"[{status}] High null columns (>30%): {high_null if high_null else 'none'}")

# 5. cart_momentum distribution
status = "OK" if cart_df['cart_momentum'].nunique() >= 3 else "WARNING"
print(f"[{status}] cart_momentum unique values: {cart_df['cart_momentum'].nunique()}")

# 6. Revenue features
print(f"\n--- Revenue Feature Distributions ---")
print(f"candidate_margin_score mean:       {cart_df['candidate_margin_score'].mean():.3f}")
print(f"candidate_revenue_potential mean:  {cart_df['candidate_revenue_potential'].mean():.4f}")
rev_pos = cart_df[cart_df['added']==1]['revenue_weighted_label'].mean()
rev_neg = cart_df[cart_df['added']==0]['revenue_weighted_label'].mean()
status = "OK" if rev_pos > rev_neg else "WARNING"
print(f"[{status}] revenue_weighted_label — pos: {rev_pos:.4f} | neg: {rev_neg:.4f} (pos > neg)")
print(f"aov_lift_if_added mean (positives): {cart_df[cart_df['added']==1]['aov_lift_if_added'].mean():.3f}")

# 7. Temporal split distribution
print(f"\n--- Temporal Split ---")
split_stats = cart_df.groupby('split').agg(
    rows=('added','count'),
    pos_rate=('added','mean')
).round(4)
for sp, row in split_stats.iterrows():
    pct = row['rows'] / len(cart_df) * 100
    status = "OK" if row['rows'] > 0 else "WARNING"
    print(f"  [{status}] {sp:6}: {row['rows']:>10,} rows ({pct:.1f}%) "
          f"| pos rate: {row['pos_rate']*100:.2f}%")

# 8. Weather features
print(f"\n--- Weather Features ---")
print(f"weather_temp_c mean: {cart_df['weather_temp_c'].mean():.1f}°C  "
      f"| range: {cart_df['weather_temp_c'].min():.1f}–{cart_df['weather_temp_c'].max():.1f}°C")
print(f"is_hot_weather rate: {cart_df['is_hot_weather'].mean()*100:.1f}%")
hot_drink = (cart_df[cart_df['is_hot_weather']==1]['added'].mean())
cold_drink = (cart_df[cart_df['is_hot_weather']==0]['added'].mean())
print(f"pos rate hot vs not-hot: {hot_drink*100:.2f}% vs {cold_drink*100:.2f}%")

# 9. Positive rate by segment
print(f"\n--- Positive Rate by Segment ---")
seg_pos = cart_df.groupby('user_segment')['added'].mean().sort_values(ascending=False)
for seg, rate in seg_pos.items():
    print(f"  {seg:22}: {rate*100:.2f}%")

# 10. Positive rate by meal slot
print(f"\n--- Positive Rate by Meal Slot ---")
slot_pos = cart_df.groupby('meal_slot')['added'].mean().sort_values(ascending=False)
for slot, rate in slot_pos.items():
    print(f"  {slot:15}: {rate*100:.2f}%")

# 11. Dataset size summary
print(f"\n--- Dataset Summary ---")
print(f"Total rows:           {len(cart_df):>12,}")
print(f"Total columns:        {len(cart_df.columns):>12}")
print(f"Unique sessions:      {cart_df['session_id'].nunique():>12,}")
print(f"Unique users:         {cart_df['user_id'].nunique():>12,}")
print(f"Unique restaurants:   {cart_df['restaurant_id'].nunique():>12,}")
print(f"Unique dish names:    {cart_df['candidate_name'].nunique():>12,}")
print(f"Positive rows:        {cart_df['added'].sum():>12,}")
print(f"Avg negatives/step:   {(1 - cart_df['added'].mean()) / cart_df['added'].mean():>11.1f}x")
print("=" * 65)


## 💾 Step 9 — Save all outputs

In [ ]:

# ═══════════════════════════════════════════════════════
# CELL: save
# DEPENDS ON: gen-users, gen-orders, gen-features, build-menu-table, gen-sessions, validate
# PRODUCES: 5 CSVs + cart_sessions.parquet + zomato_csao_dataset.zip
# ═══════════════════════════════════════════════════════

from google.colab import files
import os

OUT = '/content/zomato_csao_dataset'
os.makedirs(OUT, exist_ok=True)

# ── Small tables → CSV (fast, human-readable) ────────────────────────────────
users_df[[
    'user_id', 'city', 'segment', 'preferred_cuisines', 'orders_per_month',
    'avg_order_value', 'is_veg', 'account_age_days', 'lifetime_orders',
    'addon_rate', 'drink_rate', 'dessert_rate', 'price_sensitivity',
    'cuisine_loyalty', 'avg_items_per_order', 'preferred_slots', 'is_cold_start',
]].to_csv(f'{OUT}/users.csv', index=False)

rest_feat[[
    'restaurant_id', 'name', 'city', 'cuisine', 'price_tier',
    'rating', 'monthly_orders', 'rest_avg_margin',
]].to_csv(f'{OUT}/restaurants.csv', index=False)

menu_df.to_csv(f'{OUT}/menu_items.csv', index=False)
orders_df.to_csv(f'{OUT}/orders.csv', index=False)

with open(f'{OUT}/generate_menu.json', 'w') as f:
    json.dump(generated_menus, f, indent=2)

# ── cart_sessions → Parquet (fast I/O, ~5x smaller than CSV at this scale) ───
parquet_path = f'{OUT}/cart_sessions.parquet'
cart_df.to_parquet(parquet_path, index=False, compression='snappy')

# Also save per-split parquet files for clean train/val/test loading
for split_name in ['train', 'val', 'test']:
    split_df = cart_df[cart_df['split'] == split_name]
    split_df.to_parquet(f'{OUT}/cart_sessions_{split_name}.parquet',
                        index=False, compression='snappy')
    print(f'  {split_name}: {len(split_df):>10,} rows '
          f'| pos rate: {split_df["added"].mean()*100:.2f}%')

# ── Summary ──────────────────────────────────────────────────────────────────
parquet_mb = os.path.getsize(parquet_path) / 1e6
print(f"""
╔══════════════════════════════════════════════════════╗
║           DATASET GENERATION COMPLETE                ║
╠══════════════════════════════════════════════════════╣
║  users.csv                →  {len(users_df):>7,} rows           ║
║  restaurants.csv          →  {len(rest_feat):>7,} rows           ║
║  menu_items.csv           →  {len(menu_df):>7,} rows           ║
║  orders.csv               →  {len(orders_df):>7,} rows          ║
║  cart_sessions.parquet    →  {len(cart_df):>7,} rows          ║
╠══════════════════════════════════════════════════════╣
║  Parquet size             →  {parquet_mb:>7.1f} MB             ║
║  Unique dish names        →  {menu_df["name"].nunique():>7,}               ║
║  Positive rate            →  {cart_df["added"].mean()*100:>6.2f}%              ║
║  Unique sessions          →  {cart_df["session_id"].nunique():>7,}               ║
╚══════════════════════════════════════════════════════╝
""")

# How to load in training script:
print("# Loading in your training script:")
print("# import pandas as pd")
print("# train = pd.read_parquet('cart_sessions_train.parquet')")
print("# val   = pd.read_parquet('cart_sessions_val.parquet')")
print("# test  = pd.read_parquet('cart_sessions_test.parquet')")
print()

print("cart_sessions.parquet columns:")
for i, col in enumerate(sorted(cart_df.columns)):
    print(f"  {i+1:2}. {col}")

# Zip and download
!zip -r /content/zomato_csao_dataset.zip {OUT}
files.download('/content/zomato_csao_dataset.zip')
